# Seasonal Handling for Hydrological Flow Prediction

This notebook implements a seasonal handling that can be used as feature for other models or to use it as a benchmark model.

In [ ]:
#| default_exp season

In [ ]:
#| export
import pandas as pd

In [ ]:
#|hide
from pathlib import Path
from nbdev.showdoc import *


DATA_PATH = Path("../testing_data")
OBS_COL = 'Q_obs'

#### Data loading

In [ ]:

data = pd.read_csv(
    DATA_PATH/'hydro_example.csv', 
    usecols=['time', OBS_COL], 
    index_col='time',
    converters={"time": pd.to_datetime}
    )
data = data['2012-01-01':]

#### Seassonal model

We create a class that is able to:
- Compute seasonal patterns based on week of year
- Remove seasonality from data
- Add seasonality back to data

In [ ]:
#| export
class SeasonalityHandler:
    """Class to handle seasonality operations in time series data.
    
    This class provides methods to:
    - Compute seasonal patterns based on week of year
    - Remove seasonality from data
    - Add seasonality back to data
    
    Attributes:
        seasonal_pattern: pd.DataFrame
            The computed seasonal pattern, indexed by week of year
    """
    
    def __init__(self):
        """Initialize the SeasonalityHandler."""
        self.seasonal_pattern = None
    
    def compute_seasonal_pattern(self, data: pd.DataFrame) -> pd.DataFrame:
        """Compute mean values for each week of the year to capture seasonal patterns."""
        if 1 < len(data.columns):
            raise ValueError("Data must contain only one column")
        seasonal_pattern = data.groupby(data.index.isocalendar().week).mean()
        self.seasonal_pattern = seasonal_pattern.rename(columns={data.columns[0]: "season"})
        return self.seasonal_pattern
    
    def remove_seasonality(self, data: pd.DataFrame) -> pd.DataFrame:
        """Remove seasonality from the data."""
        if self.seasonal_pattern is None:
            raise ValueError("Seasonal pattern not computed. Call compute_seasonal_pattern first.")
        
        deseasonalized = data.copy()
        deseasonalized = self._add_week_index(deseasonalized)
        deseasonalized = deseasonalized.join(self.seasonal_pattern, on='week')
        deseasonalized = deseasonalized[data.columns] - deseasonalized["season"].values.reshape(-1, 1)
        deseasonalized = self._remove_week_index(deseasonalized)
        return deseasonalized
    
    def add_seasonality(self, data: pd.DataFrame) -> pd.DataFrame:
        """Add seasonality back to the data."""
        if self.seasonal_pattern is None:
            raise ValueError("Seasonal pattern not computed. Call compute_seasonal_pattern first.")
        
        reseasonalized = data.copy()
        reseasonalized = self._add_week_index(reseasonalized)
        reseasonalized = reseasonalized.join(self.seasonal_pattern, on='week')
        reseasonalized = reseasonalized[data.columns] + reseasonalized["season"].values.reshape(-1, 1)
        reseasonalized = self._remove_week_index(reseasonalized)
        
        return reseasonalized
    
    def _add_week_index(self, data: pd.DataFrame) -> pd.DataFrame:
        """Add week as index level to the data."""
        data['week'] = data.index.isocalendar().week
        data.set_index('week', inplace=True, append=True)
        return data
    
    def _remove_week_index(self, data: pd.DataFrame) -> pd.DataFrame:
        """Remove week as index level from the data."""
        data.reset_index(level='week', drop=True, inplace=True)
        return data
    
    def append_season(self, data: pd.DataFrame) -> pd.DataFrame:
        """Append the seasonality to the data."""
        if self.seasonal_pattern is None:
            raise ValueError("Seasonal pattern not computed. Call compute_seasonal_pattern first.")
        joint = data.copy()
        joint = self._add_week_index(joint)
        joint = joint.join(self.seasonal_pattern, on='week')
        joint = self._remove_week_index(joint)
        return joint


First we create an instance:

In [ ]:
seasonality_handler = SeasonalityHandler()

In [ ]:
show_doc(SeasonalityHandler.compute_seasonal_pattern)

Now we compute the seasonality of the data as follows:

In [ ]:
season = seasonality_handler.compute_seasonal_pattern(data)

In [ ]:
show_doc(seasonality_handler.remove_seasonality)

Its also possible to remove the seasonality to our data as follows

In [ ]:
deseasonalized_data = seasonality_handler.remove_seasonality(data)
deseasonalized_data.plot()

In [ ]:
show_doc(seasonality_handler.add_seasonality)

We can also add the seasonality back

In [ ]:
seasonality_handler.add_seasonality(deseasonalized_data).head(3)

In [ ]:
show_doc(seasonality_handler.append_season)

Or we can append the seasonal data as a new column as follows

In [ ]:
seasonality_handler.append_season(data).head(3)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()